In [ ]:
# Preliminary inputs
wandb_key = None # Insert your wandb code here

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
%cd /kaggle/working/

In [ ]:
!git clone https://github.com/KaziFahmi/EDM_Custom

In [ ]:
import sys
sys.path.append('/kaggle/working/EDM_Custom')

In [ ]:
%cd EDM_Custom/

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip install hydra-core torch_geometric wandb pytorch_lightning omegaconf torchmetrics setuptools scikit_learn imageio matplotlib numpy pandas Pillow tqdm rdkit

In [ ]:
import sys
import torch
import pytorch_lightning as pl

import os
import warnings
import pathlib
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.utilities.warnings import PossibleUserWarning

from midi.datasets import qm9_dataset, geom_dataset
from midi.diffusion_model import FullDenoisingDiffusion
from omegaconf import OmegaConf
from hydra import compose, initialize

In [ ]:
!git pull origin main

In [ ]:
!dir

In [ ]:
config_path = "EDM_Custom/configs"  
def load_hydra_configs(config_path):
    with initialize(config_path=config_path):
        cfg = compose(config_name="config")
    return cfg

In [ ]:
# %rm -rf /kaggle/working/EDM_Custom/checkpoints

In [ ]:
cfg = load_hydra_configs(config_path)

In [ ]:
dataset_config = cfg.dataset
data_module = qm9_dataset.QM9DataModule(cfg)
dataset_info = qm9_dataset.QM9infos(datamodule=data_module,cfg=cfg)

In [ ]:
model = FullDenoisingDiffusion(cfg=cfg, dataset_infos=dataset_info, train_smiles=[])

In [ ]:
params_to_ignore = ['module.model.train_smiles', 'module.model.dataset_infos']

In [ ]:
torch.nn.parallel.DistributedDataParallel._set_params_and_buffers_to_ignore_for_model(model, params_to_ignore)

In [ ]:
checkpoint_callback = ModelCheckpoint(dirpath=f"checkpoints/{cfg.general.name}",
                                              filename='{epoch}',
                                              monitor='val/epoch_NLL',
                                              save_top_k=5,
                                              mode='min',
                                              every_n_epochs=1)

In [ ]:
callbacks=[]

In [ ]:
last_ckpt_save = ModelCheckpoint(dirpath=f"checkpoints/{cfg.general.name}", filename='last', every_n_epochs=1)
callbacks.append(checkpoint_callback)
callbacks.append(last_ckpt_save)

In [ ]:
import wandb
wandb.login(key=wandb_key)

In [ ]:
trainer = Trainer(gradient_clip_val=cfg.train.clip_grad, # Needed to load old checkpoints
                      accelerator='gpu' if torch.cuda.is_available() else 'cpu',
                      devices=cfg.general.gpus,
                      max_epochs=cfg.train.n_epochs,
                      check_val_every_n_epoch=cfg.general.check_val_every_n_epochs,
                      fast_dev_run=cfg.general.name == 'debug',
                      enable_progress_bar=cfg.train.progress_bar,
                      callbacks=callbacks,
                      log_every_n_steps=50,
                      )

In [ ]:
import tracemalloc

In [ ]:
# tracemalloc.start()

In [ ]:
# FILEPATH: /f:/FYDP/EDM_Custom/colab_notebook.ipynb
trainer.fit(model, datamodule=data_module)
